In [605]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [606]:
df_pos = pd.read_excel('data/output/clean_labeled_data.xlsx')
df_neg = pd.read_excel('data/output/clean_labeled_data.xlsx')

Raghava

In [607]:
df_bit = pd.read_excel("data/output/clean_bit_data.xlsx") 

## Classifier

In [608]:
df_pos.head(10)

,COMMENT,LABEL
0,"thats, bull, say, since, pessimisticvalue, k, ...",NEG
1,"hello, guy, anyone, use, ig, market, issue, pl...",NEU
2,"year, old, hand, pin, falcon, wing, door, subm...",NEU
3,"time, breka, never, really, guys",NEG
4,"world, economy, th, final, one, trump, bankruptcy",NEG
5,"come, q, extremely, strong, ford, expect, bill...",NEG
6,"bad, pessimisticvalue",NEG
7,"see, joke, right, smell, ponzihistory, doesnt,...",NEG
8,"bear, case, touch, fall, bull, case, it, retes...",NEU
9,"distance, reentering, downward, channel, pullb...",NEG


In [609]:
df_neg.head(10)

,COMMENT,LABEL
0,"thats, bull, say, since, pessimisticvalue, k, ...",NEG
1,"hello, guy, anyone, use, ig, market, issue, pl...",NEU
2,"year, old, hand, pin, falcon, wing, door, subm...",NEU
3,"time, breka, never, really, guys",NEG
4,"world, economy, th, final, one, trump, bankruptcy",NEG
5,"come, q, extremely, strong, ford, expect, bill...",NEG
6,"bad, pessimisticvalue",NEG
7,"see, joke, right, smell, ponzihistory, doesnt,...",NEG
8,"bear, case, touch, fall, bull, case, it, retes...",NEU
9,"distance, reentering, downward, channel, pullb...",NEG


In [610]:
df_pos['LABEL'] = df_pos['LABEL'].map({'NEU':'POS','POS':'POS','NEG':'NEG'})

In [611]:
df_neg['LABEL'] = df_neg['LABEL'].map({'NEU':'NEG','POS':'POS','NEG':'NEG'})

In [612]:
df_pos.head()

,COMMENT,LABEL
0,"thats, bull, say, since, pessimisticvalue, k, ...",NEG
1,"hello, guy, anyone, use, ig, market, issue, pl...",POS
2,"year, old, hand, pin, falcon, wing, door, subm...",POS
3,"time, breka, never, really, guys",NEG
4,"world, economy, th, final, one, trump, bankruptcy",NEG


In [613]:
df_neg.head()

,COMMENT,LABEL
0,"thats, bull, say, since, pessimisticvalue, k, ...",NEG
1,"hello, guy, anyone, use, ig, market, issue, pl...",NEG
2,"year, old, hand, pin, falcon, wing, door, subm...",NEG
3,"time, breka, never, really, guys",NEG
4,"world, economy, th, final, one, trump, bankruptcy",NEG


In [614]:
def classifier(clf,strat,vec,df):
    
    y = df['LABEL'].astype(str)
    X = df.COMMENT.astype(str)
    #y = pd.get_dummies(y)
    
    if vec == "cv":
        count = CountVectorizer(binary=True,ngram_range=(1,1))
    elif vec == "tdidf":
        count = TfidfVectorizer(binary=True, ngram_range=(1,1), use_idf=False)
    
    if strat == "y":
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        X_train_counts = count.fit_transform(X_train)
        X_test_counts = count.transform(X_test)

        
    elif strat == "n":
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 
        X_train_counts = count.fit_transform(X_train)
        X_test_counts = count.transform(X_test)

    
    if clf == "lr":
        clf = LogisticRegression(solver="newton-cg",C=1).fit(X_train_counts, y_train)
        #clf = LogisticRegression(C=0.4).fit(X_train_counts, y_train)
        
    elif clf == "mnb":
        clf = MultinomialNB().fit(X_train_counts, y_train)
        
    
    elif clf == "svm":
        clf = svm.SVC(kernel='linear').fit(X_train_counts, y_train)

    yPred = clf.predict(X_test_counts)
    
    list_ = list(zip(yPred, X_test))
    
    acc=accuracy_score(y_test,yPred)
    f1=f1_score(y_test,yPred, average='weighted')

    return clf, count,"Accuracy: %s" % acc, "F1 %s" % f1, list_, y_train


In [637]:
result=classifier("lr","y","cv",df_neg)
print(result[2:4])

df_result = pd.DataFrame(result[4], columns = ['Predicted_label', 'Comment'])

C:\Users\cfo19\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


('Accuracy: 0.745', 'F1 0.712148349698332')


In [638]:
targets=df_result['Predicted_label'].values.tolist()

In [639]:
df_result

,Predicted_label,Comment
0,NEG,"hit, pessimisticvalue, morning, go, easter, al..."
1,NEG,bottom
2,NEG,"time, announcement, expect"
3,NEG,"yes, indeed, look, pessimisticvalue, percent, ..."
4,POS,"btc, attempt, stay, daily, however, pull, back..."
5,NEG,"today, point"
6,NEG,"please, dont, panic, there, tesla, space"
7,NEG,"short, folk, agree, overvalue, market, however..."
8,NEG,"wait, next, tweet, mass, destruction, potus"
9,NEG,"market, investor, lose, confidence, stock, pes..."


Trying Pipeline

In [642]:
result[0].classes_

array(['NEG', 'POS', 'nan'], dtype=object)

In [641]:
result[0].class_count_

AttributeError: 'LogisticRegression' object has no attribute 'class_count_'

In [618]:
feature_to_coef = {
    word: coef for word, coef in zip(
        result[1].get_feature_names(), result[0].coef_[0]
    )    
}
feature_to_coef

{'aamcoxxx': -8.11701408773731,
 'aapl': -8.11701408773731,
 'aback': -8.11701408773731,
 'abandon': -8.11701408773731,
 'abc': -8.810161268297255,
 'able': -8.11701408773731,
 'abruptly': -8.11701408773731,
 'absolutely': -8.11701408773731,
 'abuse': -8.11701408773731,
 'ac': -8.11701408773731,
 'accept': -7.711548979629145,
 'acceptance': -8.11701408773731,
 'accident': -8.11701408773731,
 'accommodative': -8.810161268297255,
 'accord': -7.423866907177365,
 'account': -7.711548979629145,
 'accumulate': -8.11701408773731,
 'achieve': -8.11701408773731,
 'action': -7.018401799069201,
 'activate': -8.810161268297255,
 'actual': -7.711548979629145,
 'add': -7.711548979629145,
 'added': -8.810161268297255,
 'additionally': -8.11701408773731,
 'address': -8.11701408773731,
 'advance': -7.711548979629145,
 'advise': -8.11701408773731,
 'affair': -8.11701408773731,
 'affect': -7.711548979629145,
 'afraid': -8.11701408773731,
 'afterhours': -8.11701408773731,
 'afternoon': -8.11701408773731,


In [622]:
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    )[:10]:
    print (best_positive)

('abc', -8.810161268297255)
('accommodative', -8.810161268297255)
('activate', -8.810161268297255)
('added', -8.810161268297255)
('aheadbear', -8.810161268297255)
('aheadtrget', -8.810161268297255)
('ais', -8.810161268297255)
('alarm', -8.810161268297255)
('alive', -8.810161268297255)
('allegation', -8.810161268297255)


In [623]:
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], reverse=True 
    )[:10]:
    print (best_negative)

('pessimisticvalue', -4.332824453819049)
('go', -4.667026541905723)
('tesla', -4.802828083064784)
('short', -4.98151987180816)
('market', -5.025971634378994)
('sell', -5.0489611526036935)
('buy', -5.0965892015929475)
('get', -5.12128181418332)
('know', -5.4089638866350995)
('see', -5.4089638866350995)


In [ ]:
def print_top10(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer.get_feature_names()
    for i, class_label in enumerate(class_labels):
        top10 = np.argsort(clf.coef_[i])[-10:]
        print("%s: %s" % (class_label,
              " ".join(feature_names[j] for j in top10)))

In [595]:
feature_to_coef

{'aamcoxxx': -0.08895443752014567,
 'aapl': 0.394194867688266,
 'abandon': -0.31983570594244287,
 'abc': -0.16667159664227948,
 'able': 0.13079473914059425,
 'absolutely': 0.375430302906631,
 'abuse': 0.14304070463949023,
 'acceptance': 0.07910373680516869,
 'accident': 0.16386825685915898,
 'accommodative': -0.28461454431135785,
 'accord': -0.11211708820792791,
 'account': -0.3347023962509035,
 'accumulate': -0.2980219281878312,
 'achieve': -0.26935870452522276,
 'across': -0.28532297101732895,
 'action': 0.31433017692460324,
 'activate': -0.19307504429434946,
 'actual': 0.33756060188869724,
 'actually': -0.24241903265410827,
 'add': -0.6218032937018345,
 'added': -0.17165369774642847,
 'address': -0.15947402177177522,
 'adjustment': 0.3452484557399915,
 'advance': -0.13470720417741638,
 'affair': -0.11322234562636693,
 'affect': -0.3197969690369915,
 'afraid': -0.19330446974369453,
 'afterhours': 0.22715690399822372,
 'afternoon': 0.4562375015401672,
 'afterwards': -0.224819573590844